In [1]:
import keras
from keras.models import load_model
from keras.models import model_from_json
import numpy
import os

Using TensorFlow backend.


In [2]:
file_name = 'KERAS_conv2d_model.json'
json_file = open(file_name, 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("KERAS_3layer_weights.h5")
print("Loaded model")

FileNotFoundError: [Errno 2] No such file or directory: 'KERAS_conv2d.json'

In [ ]:
loaded_model.summary()